<a href="https://colab.research.google.com/github/DotSlash-A/Pytorch/blob/main/MultiClassClassification_usingPytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
batch_size = 64

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [4]:
train = datasets.FashionMNIST(root="data",train = True,transform=ToTensor(),download=True)

100%|██████████| 26421880/26421880 [00:02<00:00, 12660563.84it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 201711.00it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3715230.16it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 5838906.70it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [24]:
test_data = datasets.FashionMNIST(root="data",train = False,transform=ToTensor(),download=True)

In [6]:
train_dataloader=DataLoader(train,batch_size)
test_dataloader=DataLoader(test_data,batch_size)


In [7]:
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [10]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten=nn.Flatten()
    self.linear=nn.Sequential(
          nn.Linear(28*28,512),
          nn.ReLU(),
          nn.Linear(512,512),
          nn.ReLU(),
          nn.Linear(512,10)
        )

  def forward(self,X):
    X=self.flatten(X)
    logits=self.linear(X)
    return logits



In [11]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [12]:
loss = nn.CrossEntropyLoss()

In [14]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [15]:
def train_function(data,model,loss,optimizer):
  model.train()
  size = len(data.dataset)
  for index, (X, y) in enumerate(data):
    X, y = X.to(device), y.to(device)
    logits=model(X)
    loss_value=loss(logits,y)
    # Backpropagation
    loss_value.backward()
    optimizer.step()
    optimizer.zero_grad()

    if index % 100 == 0:
      loss_v, current = loss_value.item(), (index + 1) * len(X)
      print(f"Loss: {loss_v:>7f}  [{current:>5d}/{size:>5d}]")



In [16]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [17]:
for i in range(10):
  print(f"Epoch {i+1}\n-------------------------------")
  train_function(train_dataloader,model,loss,optimizer)
  test(train_dataloader,model,loss)


Epoch 1
-------------------------------
Loss: 2.303382  [   64/60000]
Loss: 2.294966  [ 6464/60000]
Loss: 2.275539  [12864/60000]
Loss: 2.271707  [19264/60000]
Loss: 2.264024  [25664/60000]
Loss: 2.231974  [32064/60000]
Loss: 2.243783  [38464/60000]
Loss: 2.208949  [44864/60000]
Loss: 2.197540  [51264/60000]
Loss: 2.178336  [57664/60000]
Test Error: 
 Accuracy: 44.0%, Avg loss: 2.168692 

Epoch 2
-------------------------------
Loss: 2.173498  [   64/60000]
Loss: 2.167732  [ 6464/60000]
Loss: 2.109536  [12864/60000]
Loss: 2.127445  [19264/60000]
Loss: 2.089387  [25664/60000]
Loss: 2.024007  [32064/60000]
Loss: 2.057349  [38464/60000]
Loss: 1.977497  [44864/60000]
Loss: 1.977076  [51264/60000]
Loss: 1.918668  [57664/60000]
Test Error: 
 Accuracy: 56.8%, Avg loss: 1.907880 

Epoch 3
-------------------------------
Loss: 1.931433  [   64/60000]
Loss: 1.912718  [ 6464/60000]
Loss: 1.789177  [12864/60000]
Loss: 1.836709  [19264/60000]
Loss: 1.739893  [25664/60000]
Loss: 1.674297  [32064/600

In [18]:
torch.save(model.state_dict(),"model.pth")

In [19]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [20]:
classes = [
  "T-shirt/top",
  "Trouser",
  "Pullover",
  "Dress",
  "Coat",
  "Sandal",
  "Shirt",
  "Sneaker",
  "Bag",
  "Ankle boot",
]

In [21]:
model.eval()

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [42]:
x, y = test_data[100][0], test_data[100][1]

In [49]:
!pip install Pillow

In [51]:
import torchvision.transforms as T

In [55]:
from PIL import Image
transform = T.ToPILImage()
img = transform(x)
img.save("img.jpg")


In [54]:
img.show()

In [47]:
x

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.1373, 0.2980, 0.2824, 0.0000, 0.0000, 0.0000, 0.0000,
          0.3176, 0.2980, 0.0078, 0.0706, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.3843, 0.8118, 0.9412, 0.7137, 0.3765, 0.5098, 0.5412, 0.4235,
          0.5882, 0.7490, 0.7569, 0.6745, 0.3059, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2314,
          0.6118, 0.5882, 0.8745, 0.7608, 0.8078, 0.5294, 0.5098, 0.2588,
          0.0392, 0.3529, 0.6000, 0.7020, 0.8941, 0.1804, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.4471,
          0.6314, 0.6118, 0.8314, 0.6980, 0.7843, 0.7255, 0.2941, 0.5098,
          0.7529, 0.2471, 0.4549, 0.4314, 0.6392, 0.2275,

In [43]:
with torch.no_grad():
  x=x.to(device)
  prediction=model(x)

In [44]:
prediction

tensor([[ 1.7656,  3.2944, -0.5717,  4.3496,  0.3793, -2.0323,  0.5966, -2.5670,
         -1.8252, -2.4632]], device='cuda:0')

In [45]:
classes[prediction[0].argmax(0)]

'Dress'

In [46]:
classes[y]

'Dress'

In [36]:
print